In [ ]:
#connect to db
from pymilvus import connections, Role,db, CollectionSchema, FieldSchema, DataType, Collection
_HOST = ''
_PORT = '19530' #default port of milvus

connections.connect(host=_HOST, port=_PORT, db_name="default")
print(db.list_database())

In [ ]:
#Drop the collection
from pymilvus import utility
utility.drop_collection("milvus_final")

In [ ]:
#Create the collection
from pymilvus import CollectionSchema, FieldSchema, DataType, Collection, utility
id = FieldSchema(
  name="vector_db_id",
  dtype=DataType.INT64,
  is_primary=True,
)
title = FieldSchema(
  name="title",
  dtype=DataType.VARCHAR,
  max_length=300,
)
text = FieldSchema(
  name="text",
  dtype=DataType.VARCHAR,
  max_length=10000,
)
url = FieldSchema(
  name="url",
  dtype=DataType.VARCHAR,
  max_length=300,
)
emb = FieldSchema(
  name="emb",
  dtype=DataType.FLOAT_VECTOR,
  dim=768
)
schema = CollectionSchema(
  fields=[id, title, text, url,emb],
  description="Test book search",
  enable_dynamic_field=True
)
collection_name = "milvus_final"

collection = Collection(
    name=collection_name,
    schema=schema,
    using='default',
    shards_num=2
    )
    
index_params = {
  "metric_type":"IP",
  "index_type":"IVF_FLAT",
  "params":{"nlist":1024}
}

collection.create_index(
  field_name="emb", 
  index_params=index_params
)

utility.index_building_progress("milvus_final")

In [3]:
#Load the database from hugging face
from datasets import load_dataset
dataset = load_dataset("Cohere/wikipedia-22-12-simple-embeddings")

In [6]:
#Info about the dataset
print(len(dataset["train"]))#dataset size
print(type(dataset["train"]["id"]))

485859
<class 'list'>


In [ ]:
#Data insertion part
#It inserts the data with parts 1000 entities in each to reduce the lag and fast and contunious insertion
import random
collection = Collection("milvus_final") # Get an existing collection.

data=dataset["train"]
ids=data["id"]
titles=data["title"]
texts=data["text"]
urls=data["url"]
ran=1000 #size of each insertion
for a in range(0,420000-ran,ran):#I didn't insert all the data because it makes the server crash. If you want to insert all comment this line and uncomment the following line
#for a in range(0,len(data)-ran,ran)
  embeeding=[]
  for i in range(a,a+ran):
      d=dataset["train"][i]["emb"]
      embeeding.append(d)

  data = [
    ids[a:a+ran],#id
    titles[a:a+ran],#title
    texts[a:a+ran],#text
    urls[a:a+ran],#url
    embeeding
  ]
  mr = collection.insert(data)
  print(a," ",mr)#prints the index to track the process